<a href="https://colab.research.google.com/github/malihasaeed/RAG/blob/main/Copy_of_RAG_assignment1_atomcamp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U langchain
# Requires Python 3.10+

In [ ]:
!pip install -U groq langchain-groq


In [ ]:
!pip install -qU \
    langchain \
    langchain-community \
    langchain-core \
    langchain-groq \
    faiss-cpu \
    pypdf \
    gradio

In [ ]:
import os
os.environ["GROQ_API_KEY"] = "rag_chatbot"


In [ ]:
#This code saves your secret API key, then creates a smart-robot helper (llm) so you can ask it questions.

from google.colab import userdata
import os

# read your secret by its name
groq_key = userdata.get("rag_chatbot")

# set the official env var Groq expects
os.environ["GROQ_API_KEY"] = groq_key

from langchain_groq import ChatGroq

llm = ChatGroq(model="llama-3.3-70b-versatile")

# llm.invoke(...) is a function (a built-in action) that sends your question to the AI and brings back the answer, which is then printed
response = llm.invoke("Explain retrieval-augmented generation in simple words.")
#.content pulls out ONLY the words the AI wrote so you can print them.
print(response.content)


In [ ]:
from google.colab import files
uploaded = files.upload()


In [ ]:
#This is standard Python code commonly used in tutorials / documentation to read PDFs using PyPDF2.
#It’s not specific to any AI tool.
#converting the doc into text
!pip install PyPDF2

import PyPDF2

pdf_file = open("business proposal.pdf", "rb")
reader = PyPDF2.PdfReader(pdf_file)

text = ""

for page in reader.pages:
    text += page.extract_text()

pdf_file.close()

print(text[:500])  # show first 500 chars


In [ ]:
import langchain
print(langchain.__version__)


In [ ]:
import langchain
print(langchain.__version__)


In [ ]:
!pip install -q langchain langchain-text-splitters langchain-core


what logic is being used?

🔹 PDFs can be VERY long

🔹 LLMs cannot read huge text at once

🔹 So we break it into small readable pieces (chunks)

This makes RAG / chatbots work better and faster 🚀

In [ ]:
#This is a tool from LangChain that breaks long text into smaller pieces (chunks).
from langchain_text_splitters import RecursiveCharacterTextSplitter
#This wraps your text into a Document object — a neat container LangChain likes to work with.text=documemt box
from langchain_core.documents import Document

# Wrap your extracted text into a Document
#Creates a document that stores your PDF text
docs = [Document(page_content=text)]

# Create the splitter
#each text piece can have up to 800 characters
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=200,
)

# Split into chunks
split_docs = text_splitter.split_documents(docs)

print("Total chunks:", len(split_docs))
print("First chunk:\n", split_docs[0].page_content[:500])


In [ ]:
!pip install -q langchain-community faiss-cpu sentence-transformers


You are creating an object called embeddings.

Inside, you tell it which model to use:
"sentence-transformers/all-MiniLM-L6-v2".

What this object does:

You give it a sentence or chunk of text →

It returns a vector (a long list of numbers) that represents the meaning of that text.

example:
Imagine each sentence is a student, and embeddings give each student a score-card of numbers describing who they are. Similar students → similar score-cards.

In [ ]:
#Import embeddings + FAISS and build the index
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

# Create an embeddings model (lightweight, good for Colab)
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

# Build the FAISS vector store from your chunks
#Stores all those vectors inside a FAISS database → vectorstore
vectorstore = FAISS.from_documents(split_docs, embeddings)

# Turn it into a retriever (k = how many chunks to fetch per question)
retriever = vectorstore.as_retriever(search_kwargs={"k": 4})

print("Vector store built! ✅")


In [ ]:
!pip install -q langchain
!pip install -qU "langchain>=0.3" "langchain-community>=0.3" langchain-core


In [ ]:
!pip install -qU "langchain==0.2.14" "langchain-community==0.2.12"


In [ ]:
# Simple in-memory chat history(object)
#It creates a simple list named chat_history to store past messages as (role, text) pairs so the program can remember the conversation.
chat_history = []  # list of (role, text) tuples, e.g. ("user", "..."), ("assistant", "...")


In [ ]:
#Helper to format chat history for the prompt
def format_chat_history(history):
    if not history:
        return "No previous conversation."
    lines = []
    for role, text in history:
        lines.append(f"{role.capitalize()}: {text}")
    return "\n".join(lines)


In [ ]:
#Define system instructions for the assistant

SYSTEM_INSTRUCTIONS = """
You are a helpful Product Support Assistant for a single business proposal document.
You must answer ONLY using the information from the provided context.

If the answer is NOT clearly supported by the context, say exactly:
"I couldn’t find this information in the available documents."

Be concise and clear.
"""


In [ ]:
def answer_with_rag(question: str) -> str:
    global chat_history

    # 1) Retrieve relevant chunks directly from the FAISS vector store
    #    instead of using retriever.get_relevant_documents(...)
    docs = vectorstore.similarity_search(question, k=4)

    if not docs:
        answer = "I couldn’t find this information in the available documents."
        chat_history.append(("user", question))
        chat_history.append(("assistant", answer))
        return answer

    context_text = "\n\n".join(d.page_content for d in docs)

    # 2) Prepare chat history text for the prompt
    history_text = format_chat_history(chat_history)

    # 3) Build the full prompt
    prompt = f"""{SYSTEM_INSTRUCTIONS}

Context from the document:
{context_text}

Chat history:
{history_text}

User question: {question}

Answer:"""

    # 4) Call your Groq LLM
    response = llm.invoke(prompt)   # ChatGroq -> AIMessage
    try:
        answer = response.content
    except AttributeError:
        answer = str(response)

    # 5) Update simple memory
    chat_history.append(("user", question))
    chat_history.append(("assistant", answer))

    return answer


In [ ]:
print(answer_with_rag("What is the main objective of this business proposal?"))


In [ ]:
print(answer_with_rag("what do you offer"))

In [ ]:
# 1) Global chat history
chat_history = []

# 2) The helper function
def format_chat_history(history):
    if not history:
        return "No previous conversation."
    lines = []
    for role, text in history:
        lines.append(f"{role.capitalize()}: {text}")
    return "\n".join(lines)

# 3) System instructions
SYSTEM_INSTRUCTIONS = """
You are a helpful Product Support Assistant for a single business proposal document.
You must answer ONLY using the information from the provided context.

If the answer is NOT clearly supported by the context, say exactly:
"I couldn’t find this information in the available documents."

Be concise and clear.
"""

# 4) Your Groq model
# llm = ChatGroq(...)

# 5) vectorstore built from your chunks
# vectorstore = FAISS.from_documents(split_docs, embeddings)

# 6) The latest version of answer_with_rag using vectorstore.similarity_search
def answer_with_rag(question: str) -> str:
    global chat_history

    docs = vectorstore.similarity_search(question, k=4)

    if not docs:
        answer = "I couldn’t find this information in the available documents."
        chat_history.append(("user", question))
        chat_history.append(("assistant", answer))
        return answer

    context_text = "\n\n".join(d.page_content for d in docs)
    history_text = format_chat_history(chat_history)

    prompt = f"""{SYSTEM_INSTRUCTIONS}

Context from the document:
{context_text}

Chat history:
{history_text}

User question: {question}

Answer:"""

    response = llm.invoke(prompt)
    try:
        answer = response.content
    except AttributeError:
        answer = str(response)

    chat_history.append(("user", question))
    chat_history.append(("assistant", answer))

    return answer


In [ ]:
print(answer_with_rag("what do you offer"))

In [ ]:
!pip install -q gradio


In [ ]:
import gradio as gr

def gradio_qa(message):
    """
    Simple wrapper: takes user text and returns the RAG answer.
    Conversation memory is handled inside answer_with_rag via chat_history.
    """
    return answer_with_rag(message)

demo = gr.Interface(
    fn=gradio_qa,
    inputs=gr.Textbox(lines=2, label="Ask about the business proposal"),
    outputs=gr.Textbox(label="Assistant"),
    title="Business Proposal Support Assistant",
    description="Ask questions about the business proposal PDF. Answers come only from that document.",
)

demo.launch(share=False)
